## 6.3 SQLite - podstawy

Napiszmy przykładową aplikację wykorzystującą bazę `SQLite` - w tym przykładzie nie będziemy wykonywać operacji na bazie asynchronicznie. Zaimplementujemy podstawowe operacje CRUD. Aplikacja będzie wykorzystywać pojedynczą aktywność.

<table><tr><td><img src="https://media3.giphy.com/media/FIcgLsiTwUhfuSjM8X/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/04Ir1MiMp02l2U1GO6/giphy.gif" width="200" /></td></tr></table>

### **Podstawy aplikacji**

Rozpocznijmy od dodania `ViewBinding` - do pliku `build.gradle(Module)` dodaję

```kotlin
android {
    ...
    buildFeatures {
        viewBinding = true
    }
}
```

Następnie napiszmy layout głównej aktywności w postaci prostego formularza.

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".MainActivity">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:orientation="horizontal">

        <TextView
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.25"
            android:text="Name"
            android:textSize="18sp"/>

        <EditText
            android:id="@+id/edit_text_name"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.75"
            android:hint="Enter Name" />

    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:orientation="horizontal">

        <TextView
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.25"
            android:text="Index Number"
            android:textSize="18sp"/>

        <EditText
            android:id="@+id/edit_text_index"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.75"
            android:hint="Enter Index Number"
            android:inputType="number"
            android:importantForAutofill="no" />

    </LinearLayout>

    <Button
        android:id="@+id/add_button"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_margin="10dp"
        android:text="ADD STUDENT"/>

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/dataBase_recycler_view"
        android:layout_width="match_parent"
        android:layout_height="match_parent"/>
</LinearLayout>
```

Będziemy wykorzystywać `RecyclerView` do wyświetlania danych z bazy, więc dodajmy layout dla pojedynczego elementu

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/linear_layout_main"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:gravity="center_vertical"
    android:orientation="horizontal"
    android:padding="10dp">

    <TextView
        android:id="@+id/text_view_id"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:textColor="@android:color/black"
        android:textSize="16sp"
        android:padding="10dp"
        tools:text="1" />

    <TextView
        android:id="@+id/text_view_name"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:textColor="@android:color/black"
        android:textSize="16sp"
        tools:text="Name" />

    <TextView
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="5dp"
        android:layout_marginEnd="5dp"
        android:textSize="18sp"
        android:textStyle="bold"
        tools:text=":" />

    <TextView
        android:id="@+id/text_view_index"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:layout_weight="1"
        android:textColor="@android:color/black"
        android:textSize="16sp"
        tools:text="Index" />

    <ImageView
        android:id="@+id/image_view_edit"
        android:layout_width="50dp"
        android:layout_height="50dp"
        android:contentDescription="image"
        android:foreground="?attr/selectableItemBackgroundBorderless"
        android:scaleType="center"
        android:src="@drawable/ic_action_edit_foreground" />

    <ImageView
        android:id="@+id/image_view_delete"
        android:layout_width="50dp"
        android:layout_height="50dp"
        android:contentDescription="image"
        android:foreground="?attr/selectableItemBackgroundBorderless"
        android:scaleType="center"
        android:src="@drawable/ic_action_delete_foreground" />
</LinearLayout>
```

Przy edycji będziemy wykorzystywać `Dialog`, więc również tutaj stwórzmy layout

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="300dp"
    android:layout_height="wrap_content"
    xmlns:tools="http://schemas.android.com/tools"
    android:orientation="vertical"
    android:padding="10dp"
    tools:context=".MainActivity">

    <TextView
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Update Student Data"
        android:textColor="@android:color/black"
        android:textSize="18sp"
        android:textStyle="bold" />

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:orientation="horizontal">

        <TextView
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.25"
            android:text="Name"
            android:textSize="18sp" />

        <EditText
            android:id="@+id/edit_text_name_update"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.75"
            android:hint="Enter Name" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:orientation="horizontal">

        <TextView
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.25"
            android:text="Index"
            android:textSize="18sp" />

        <EditText
            android:id="@+id/edit_text_index_update"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.75"
            android:inputType="textEmailAddress"
            android:hint="Enter Index Number" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="10dp"
        android:gravity="end"
        android:orientation="horizontal">

        <Button
            android:id="@+id/button_update"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:gravity="center"
            android:padding="10dp"
            android:text="UPDATE"
            android:textStyle="bold" />

        <Button
            android:id="@+id/button_cancel"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:gravity="center"
            android:padding="10dp"
            android:text="CANCEL"
            android:textStyle="bold" />

    </LinearLayout>
</LinearLayout>
```

Dodajmy również model danych

In [ ]:
data class Student(val name: String, val index: Int) {
    var id: Int = 0

    constructor(id: Int, name: String, index: Int) : this(name, index) {
        this.id = id
    }
}

### **DBHelper**

Przejdźmy do utworzenia bazy, zrobimy to w klasie o nazwie (standardowo) `dbHelper`, będzie ona rozszerzać klasę `SQLiteOpenHelper` - przyjmuje ona w konstruktorze szereg parametrów.
- `context`
- `name: String` - nazwę bazy
- `SQLiteDatabase.CursorFactory` - wymagana gdy wykorzystujemy podklasy klasy `Cursor` do wykonywania zapytań - w tym przykładzie nie będziemy z niego korzystać, więc podajemy `null`
- `version: Int` - wersja bazy

Podstawowe informacje o bazie będziemy przechowywać w `companion object`

In [ ]:
class DBHandler(context: Context) : SQLiteOpenHelper(
    context, DATABASE_NAME, null, DATABASE_VERSION
) {
    private companion object{
        private const val DATABASE_VERSION = 1
        private const val DATABASE_NAME = "studentsDBKotlin.db"
        private const val TABLE_STUDENTS = "StudentTable"

        private const val COLUMN_ID = "_id"
        private const val COLUMN_NAME = "name"
        private const val COLUMN_INDEX = "indexNumber"
    }

    override fun onCreate(db: SQLiteDatabase?) {
        TODO("Not yet implemented")
    }

    override fun onUpgrade(db: SQLiteDatabase?, oldVersion: Int, newVersion: Int) {
        TODO("Not yet implemented")
    }
}

Musimy zaimplementować dwie metody
- `onCreate` - wywoływana gdy plik o zadanej nazwie (`BADABASE_NAME`) nie istnieje
- `onUpgrade` - wywoływana gdy schemat bazy uległ zmianie

Rozpocznijmy od metody `onCreate`. W parametrze metody dostajemy dostęp do `SQLiteDatabase`, na tej zmiennej wykonujemy instrukcje wywołując metodę `execSQL`

In [ ]:
override fun onCreate(db: SQLiteDatabase?) {
    val CREATE_STUDENTS_TABLE =
        "CREATE TABLE $TABLE_STUDENTS(" +
                "$COLUMN_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL," +
                "$COLUMN_NAME TEXT," +
                "$COLUMN_INDEX INTEGER)"
    db?.execSQL(CREATE_STUDENTS_TABLE)
}

W metodzie `onUpgrade` usuniemy bazę i wywołamy metodę `onCreate`

In [ ]:
override fun onUpgrade(db: SQLiteDatabase?, oldVersion: Int, newVersion: Int) {
    db?.execSQL("DROP TABLE IF EXISTS $TABLE_STUDENTS")
    onCreate(db)
}

Zaimplementujmy kilka metod umożliwiających przeprowadzenie podstawowych operacji na bazie.

#### **ADD**

Zacznijmy od metody `addStudent`

In [ ]:
fun addStudent(student: Student){}

W pierwszej kolejności musimy uzyskać dostęp do bazy w celu zapisu - robimy to wywołując metodę `getWritableDatabase`

In [ ]:
val db = this.writableDatabase

Następnie przygotujmy dane do dodaniam w tym celu wykorzystujemy `ContentValues` - tworzy zestaw wartości dla `ContentResolver`

In [ ]:
val contentValues = ContentValues()
contentValues.put(COLUMN_NAME, student.name)
contentValues.put(COLUMN_INDEX, student.index)

Następnie wykonujemy metodę `insert` na bazie, przyjmuje ona trzy argumenty
- nazwę tabeli do której wykonana jest operacja
- `nullColumnHack` - umożliwia wstawienie pustej wartości
- `ContentValues` - dane do wstawienia

In [ ]:
db.insert(TABLE_STUDENTS, null, contentValues)

Na koniec zamykamy dostęp do bazy wywołując metodę `close`

In [ ]:
db.close()

Pełna metoda `addStudent`

In [ ]:
fun addStudent(student: Student){
    val db = this.writableDatabase

    val contentValues = ContentValues()
    contentValues.put(COLUMN_NAME, student.name)
    contentValues.put(COLUMN_INDEX, student.index)

    db.insert(TABLE_STUDENTS, null, contentValues)
    db.close()
}

#### **DELETE**

Przejdźmy do metody umożliwiającej usunięcie wpisu. Rozpoczynamy jak poprzednio od uzyskania dostępu do bazy w celu zapisu.

In [ ]:
fun deleteStudent(student: Student){
    val db = this.writableDatabase

Następnie wywołujemy metodę `delete` na bazie - przyjmuje ona kilka argumentów
- nazwę tabeli z której chcemy usunąć wpis
- `whereClause` - klauzula warunkowa - tutaj chcemy usunąć element o zadanym `id`, więc podajemy 
```kotlin
`COLUMN_ID + "=" + student._id`
```
- `whereArgs` - argumenty klauzuli `where`

In [ ]:
db.delete(
    TABLE_STUDENTS,
    "$COLUMN_ID=${student.id}",
    null)

Ostatnim krokiem jest zamknięcie dostępu do bazy

In [ ]:
db.close()

#### **UPDATE**

Metoda aktualizacji wygląda podobnie jak poprzednie. `update` przyjmuje trzy parametry
- `ContentValues`
- `whereClause`
- `whereArgs`

In [ ]:
fun updateStudent (id: Int, name: String, index: Int){
    val db = this.writableDatabase
    
    val contentValues = ContentValues()
    contentValues.put(COLUMN_NAME, name)
    contentValues.put(COLUMN_INDEX, index)

    db.update(TABLE_STUDENTS,
        contentValues,
        "$COLUMN_ID=$id",
        null)

    db.close()
}

#### **GET**

Pozostaje metoda zwracająca listę wszystkich elementów w bazie.

In [ ]:
fun getStudents(): List<Student> {}

W metodzie stworzymy mutowalną listę do której dodamy wszystkie elementy z bazy.

In [ ]:
val students: MutableList<Student> = ArrayList()

Następnie otworzymy bazę tylko do odczytu

In [ ]:
val db = this.readableDatabase

Wynik zapytania sql zwracamy do obiektu typu `Cursor`

In [ ]:
val cursor = db.rawQuery("SELECT * FROM $TABLE_STUDENTS", null)

Metoda `rawQuery` przyjmuje dwa argumenty
- zapytanie jako `String`
- `selectionArgs` - tablica wszystkich argumentów

Następnie sprawdzamy czy `Cursor` nie jest pusty - wykonujemy to wywołując metodę `moveToFirst`. Przesuwa ona `Cursor` do pierwszego rzędu, zwraca `false` gdy `Cursor` jest pusty

In [ ]:
if (cursor.moveToFirst())

W warunku `if` odczytujemy wartości i dodajemy je na listę `students` dopóki `Cursor` posiada wartości.

In [ ]:
if (cursor.moveToFirst()) {
    do {
        students.add(Student(
                cursor.getInt(0), 
                cursor.getString(1), 
                cursor.getInt(2)))
    } while (cursor.moveToNext())
}

W bazie mamy trzy kolumny (`id: Int`, `name: String`, `index: Int`), z `Cursor` wyciągamy trzy wartości (**ważna kolejność**) i używamy ich jako argumenty konstruktora klasy `Student`.

Ostatnim elementem jest zamknięcie dostępu do bazy oraz cursora

In [ ]:
db.close()
cursor.close()

Pełny kod metody

In [ ]:
fun getStudents(): List<Student> {
    val students: MutableList<Student> = ArrayList()

    val db = this.readableDatabase

    val cursor = db.rawQuery("SELECT * FROM $TABLE_STUDENTS", null)

    if (cursor.moveToFirst()) {
        do {
            students.add(Student(
                    cursor.getInt(0),
                    cursor.getString(1),
                    cursor.getInt(2)))
        } while (cursor.moveToNext())
    }

    db.close()
    cursor.close()
    return students
}

### **Adapter**

Przejdźmy do `StudentAdapter`, w konstruktorze przekażemy `context` oraz `DBHandler`

In [ ]:
class StudentAdapter(private val dbHandler: DBHandler, private val context: Context)

W metodzie `onCreateViewHolder` wykorzystamy `ViewBinding`

In [ ]:
override fun onCreateViewHolder(
    parent: ViewGroup,
    viewType: Int
): ViewHolder {
    val itemBinding = ItemRowBinding.inflate(
        LayoutInflater.from(parent.context), parent, false)
    return ViewHolder(itemBinding)
}

W metodzie `onBindViewHolder` wywołujemy funkcję `bind` klasy `ViewHolder`

In [ ]:
override fun onBindViewHolder(holder: ViewHolder, position: Int) {
    val item = dbHandler.getStudents()[position]
    holder.bind(item)
}

Metoda `getItemCount` zawiera rozmiar całej bazy

In [ ]:
override fun getItemCount() = dbHandler.getStudents().size

Przejdźmy do klasy `ViewHolder`

In [ ]:
class StudentAdapter(private val dbHandler: DBHandler, private val context: Context) :
    RecyclerView.Adapter<StudentAdapter.ViewHolder>() {

    inner class ViewHolder(private val itemBinding: ItemRowBinding) :
        RecyclerView.ViewHolder(itemBinding.root) {
        }
}

W metodzie `bind` w epirwszej kolejności ustawiamy pola `TextView`

In [ ]:
fun bind(item: Student) {
    itemBinding.textViewName.text = item.name
    itemBinding.textViewIndex.text = item.index.toString()
    itemBinding.textViewId.text = item.id.toString()
}

Następnie obsłużmy metodę `onClick` dla `ImageView` odpowiadającego za usuwanie elementów

In [ ]:
itemBinding.imageViewDelete.setOnClickListener {
    dbHandler.deleteStudent(item)
    notifyItemRemoved(adapterPosition)
}

Następnie obsłużymy zdarzenie `onClick` pola `ImgaView` odpowiedzialnego za aktualizację wpisu, wywołamy w niej metodę `setupDialog` którą następnie zaimplementujemy

In [ ]:
itemBinding.imageViewEdit.setOnClickListener {setupDialog(item) }

Pełna metoda metoda `bind`

In [ ]:
fun bind(item: Student) {
    itemBinding.textViewName.text = item.name
    itemBinding.textViewIndex.text = item.index.toString()
    itemBinding.textViewId.text = item.id.toString()

    itemBinding.imageViewDelete.setOnClickListener {
        dbHandler.deleteStudent(item)
        notifyItemRemoved(adapterPosition)
    }

    itemBinding.imageViewEdit.setOnClickListener {setupDialog(item) }
}

Następnie do klasy `ViewHolder` dodajmy metodę tworzącą `Dialog` który posłuży do edycji. Również tutaj wykorzystamy `ViewBinding`

In [ ]:
private fun setupDialog(item: Student){
    val dialog = Dialog(context)
    val dialogBinding = DialogUpdateBinding.inflate(LayoutInflater.from(context))
    dialog.apply {
        setCancelable(false)
        setContentView(dialogBinding.root)
    }

`setCancelable` ustawiony na `false` wyłączy domyślny przycisk `Cancel`. Następnie obsłużymy wszystkie elementy `Dialog`

In [ ]:
dialogBinding.apply {
    editTextIndexUpdate.setText(item.index.toString())
    editTextNameUpdate.setText(item.name)
    buttonUpdate.setOnClickListener {
        updateDialog(dialogBinding, item, dialog)
    }

    buttonCancel.setOnClickListener { dialog.dismiss() }
}
dialog.show()

W obsłudze zdarzenia `onClick` przycisku `Update` wywołujemy metodę `updateDialog` przyjmującą trzy argumenty. Zaimplementujmy ją - chcemy wykonać aktualizację wpisu w bazie na podstawie zawartości pól `EditText`, w pierwszej kolejności sprawdzimy czy pola nie są puste

In [ ]:
private fun updateDialog(
    dialogBinding: DialogUpdateBinding,
    item: Student,
    dialog: Dialog
) {
    val updateName = dialogBinding.editTextNameUpdate.text.toString()
    val updateIndex = dialogBinding.editTextIndexUpdate.text.toString()

    if (updateName.isNotEmpty() && updateIndex.isNotEmpty()) {}

Następnie wykonujemy `update` z klasy `dbHandler`, powiadamiamy adapter o zmianie i wyłączamy dialog

In [ ]:
if (updateName.isNotEmpty() && updateIndex.isNotEmpty()) {
    dbHandler.updateStudent(item.id, updateName, updateIndex.toInt())
    notifyItemChanged(item.id - 1)
    dialog.dismiss()
}

Pełny kod klasy `ViewHolder`

In [ ]:
inner class ViewHolder(private val itemBinding: ItemRowBinding) :
    RecyclerView.ViewHolder(itemBinding.root) {
    fun bind(item: Student) {
        itemBinding.textViewName.text = item.name
        itemBinding.textViewIndex.text = item.index.toString()
        itemBinding.textViewId.text = item.id.toString()

        itemBinding.imageViewDelete.setOnClickListener {
            dbHandler.deleteStudent(item)
            notifyItemRemoved(item.id - 1)
        }

        itemBinding.imageViewEdit.setOnClickListener {setupDialog(item) }
    }

    private fun setupDialog(item: Student){
        val dialog = Dialog(context)
        val dialogBinding = DialogUpdateBinding.inflate(LayoutInflater.from(context))
        dialog.apply {
            setCancelable(false)
            setContentView(dialogBinding.root)
        }

        dialogBinding.apply {
            editTextIndexUpdate.setText(item.index.toString())
            editTextNameUpdate.setText(item.name)
            buttonUpdate.setOnClickListener {
                updateDialog(dialogBinding, item, dialog)
            }

            buttonCancel.setOnClickListener { dialog.dismiss() }
        }
        dialog.show()
    }

    private fun updateDialog(
        dialogBinding: DialogUpdateBinding,
        item: Student,
        dialog: Dialog
    ) {
        val updateName = dialogBinding.editTextNameUpdate.text.toString()
        val updateIndex = dialogBinding.editTextIndexUpdate.text.toString()

        if (updateName.isNotEmpty() && updateIndex.isNotEmpty()) {
            dbHandler.updateStudent(item.id, updateName, updateIndex.toInt())
            notifyItemChanged(item.id - 1)
            dialog.dismiss()
        }
    }
}

### **Aktywność**

Dodajmy `ViewBinding`

In [ ]:
private val binding by lazy { ActivityMainBinding.inflate(layoutInflater) }

override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    setContentView(binding.root)
}

Zainicjujmy `DBHandler` oraz zamknijmy dostęp do bazy w metodzie `onDestroy`

In [ ]:
class MainActivity : AppCompatActivity() {
    private val dbHandler by lazy { DBHandler(this) }
    ...
    
    override fun onDestroy() {
        dbHandler.close()
        super.onDestroy()
    }
}

Dodajmy `RecyclerView` w metodzie `onCreate`

In [ ]:
binding.dataBaseRecyclerView.apply {
    layoutManager = LinearLayoutManager(this@MainActivity)
    adapter = StudentAdapter(dbHandler, this@MainActivity)
}

Następnie obsłużmy zdarzenie `onClick` przycisku odpowiadającego za dodawanie elementów do bazy. Wpierw sprawdzamy czy pola `EditText` nie są puste, następnie wywołujemy metodę `addStudent` klasy `DBHandler`. Na koniec czyścimy pola `EditText` oraz powiadamiamy adapter o dodaniu nowej pozycji.

In [ ]:
binding.addButton.setOnClickListener {
    val name = binding.editTextName.text.toString()
    val index = binding.editTextIndex.text.toString()

    if (name.isNotEmpty() && index.isNotEmpty()){
        dbHandler.addStudent(Student(name, index.toInt()))
        binding.editTextName.text.clear()
        binding.editTextIndex.text.clear()
    }

    binding.dataBaseRecyclerView.adapter?.notifyItemInserted(dbHandler.getStudents().size)
}

Możemy przetestować aplikację

<table><tr><td><img src="https://media3.giphy.com/media/FIcgLsiTwUhfuSjM8X/giphy.gif" width="150" /></td><td><img src="https://media2.giphy.com/media/04Ir1MiMp02l2U1GO6/giphy.gif" width="150" /></td></tr></table>